In [1]:
import pyslha,itertools
import logging as logger

In [27]:
def equalObjs(obj1,obj2,rtol=0.01,ignore=['decays']):
    
    if type(obj1) != type(obj2):
        logger.warning("Data types differ (%s,%s)" %(type(obj1),type(obj2)))
        return False
   
    if type(obj1) in [ float, int ]:
        obj1,obj2=float(obj1),float(obj2)
    
    if isinstance(obj1,list):
        obj1 = sorted(obj1)
        obj2 = sorted(obj2)
    
    if obj1 == obj2:
        return True

    if isinstance(obj1,float):
        if obj1 == obj2:
            return True
        diff = 2.*abs(obj1-obj2)/abs(obj1+obj2)
        return diff < rtol
    elif isinstance(obj1,str):
        return obj1.lower() == obj2.lower()
    elif isinstance(obj1,dict):
        for key in obj1:
            if key in ignore:
                continue
            if not key in obj2:
                logger.warning("Key %s missing" %str(key))
                return False
            if not equalObjs(obj1[key],obj2[key],rtol, ignore=ignore ):
                logger.warning( "Dictionaries differ in key %s" %str(key))
                s1,s2 = str(obj1[key]),str(obj2[key]) 
                if len(s1) + len(s2) > 200:
                    logger.warning ( "The values are too long to print." )
                else:
                    logger.warning( 'The values are: %s versus %s'%\
                                ( s1,s2 ) )
                return False
    elif isinstance(obj1,list):
        if len(obj1) != len(obj2):
            logger.warning('Lists differ in length:\n   %i (this run)\n and\n   %i (default)' %\
                                (len(obj1),len(obj2)))
            return False
        for ival,val in enumerate(obj1):
            if not equalObjs(val,obj2[ival],rtol):
                logger.warning('Lists differ:\n   %s (this run)\n and\n   %s (default)' %\
                                (str(val),str(obj2[ival])))
                return False
    else:
        if not equalObjs(obj1.__dict__,obj2.__dict__):
            return False
        
    return True

In [28]:
#Load the SLHA data
slhaFiles = {'andre' : './lhiggs_slha/lhiggs_F500_13.slha', 'juhi' : './juhi_files/LHT500GeV.slha'}
slhaData = dict([[key,pyslha.readSLHAFile(val)] for key,val in slhaFiles.items()])

In [29]:
#Loop over the files and compare them 2 at a time:
for file1,file2 in itertools.product(slhaData.keys(),slhaData.keys()):
    if file1 == file2:
        continue
    slha1 = slhaData[file1]
    slha2 = slhaData[file2]
    comp = equalObjs(slha1,slha2,rtol=0.01)
    if not comp:
        print('Files %s and %s differ' %(file1,file2))
    else:
        print('Files are equal')

  CKMBLOCK
  LHTBLOCK
  MASS
  SMINPUTS
  YUKAWA
  QNUMBERS versus Blocks
  CKMBLOCK
  LHTBLOCK
  MASS
  SMINPUTS
  YUKAWA
  QNUMBERS
  CKMBLOCK
  LHTBLOCK
  MASS
  SMINPUTS
  YUKAWA
  QNUMBERS versus Blocks
  CKMBLOCK
  LHTBLOCK
  MASS
  SMINPUTS
  YUKAWA
  QNUMBERS


Files andre and juhi differ
Files juhi and andre differ


In [32]:
print(sorted(slhaData['andre'].blocks.keys()) ==sorted(slhaData['juhi'].blocks.keys()))

True


In [18]:
print(len(slhaData['juhi'].xsections))
print(len(slhaData['andre'].xsections))

106
106


In [19]:
diffs = []
for pdg in slhaData['juhi'].decays:
    decA = slhaData['juhi'].decays[pdg]
    decB = slhaData['andre'].decays[pdg]
    if decA != decB:
        print(pdg)
        print(decA)
        print(decB)
        print('\n')

8880001
8880001 : mass = 1.41e+03 GeV : total width = 19.4 GeV
   0.62 [-8880024, 2]
   0.3 [8880023, 1]
   0.063 [8880022, 1]
   0.016 [8880023, 3]
8880001 : mass = 1.41e+03 GeV : total width = 19.5 GeV
   0.62 [-8880024, 2]
   0.3 [8880023, 1]
   0.063 [8880022, 1]
   0.016 [8880023, 3]


8880002
8880002 : mass = 1.4e+03 GeV : total width = 19.4 GeV
   0.58 [8880024, 1]
   0.3 [8880023, 2]
   0.09 [8880022, 2]
   0.031 [8880024, 3]
8880002 : mass = 1.4e+03 GeV : total width = 19.4 GeV
   0.58 [8880024, 1]
   0.3 [8880023, 2]
   0.09 [8880022, 2]
   0.031 [8880024, 3]


8880003
8880003 : mass = 1.41e+03 GeV : total width = 19.4 GeV
   0.62 [-8880024, 4]
   0.3 [8880023, 3]
   0.063 [8880022, 3]
   0.016 [8880023, 1]
8880003 : mass = 1.41e+03 GeV : total width = 19.5 GeV
   0.62 [-8880024, 4]
   0.3 [8880023, 3]
   0.063 [8880022, 3]
   0.016 [8880023, 1]


8880004
8880004 : mass = 1.4e+03 GeV : total width = 19.4 GeV
   0.58 [8880024, 3]
   0.3 [8880023, 4]
   0.09 [8880022, 4]
   0.0

In [ ]:
print(min(diffs),max(diffs))